In [2]:
import streamlit as st 
from scrapper import fk_scrapper_functions as sc
from scrapper import flipkart_json_scrapper_with_all_specifications as fk_scrapper
import time
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
competitor_data=pd.read_pickle('competitor_data.pkl')

d:\personal_git\data science\portfolio projects\Competition analysis\.venv\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
competitor_data=pd.read_pickle('competitor_data.pkl')
st.write(competitor_data['all_specs'][0].get('Model Name'))
competitor_data['brand']=competitor_data['title'].str.split(" ").str[0]
brand_level_data=competitor_data.copy()
st.dataframe(brand_level_data)
for index, row in brand_level_data.iterrows():
    specs = row['all_specs']
    for key, value in specs.items():
        if key not in brand_level_data.columns:
            brand_level_data[key] = None
        brand_level_data.at[index, key] = value
most_important_feature={}
feature_name=[]
feature_count=[]
for column in brand_level_data.columns:
    feature_name.append(column)
    feature_count.append(brand_level_data[column].count())
most_important_feature['feature_name']=feature_name
most_important_feature['feature_count']=feature_count
important_features=pd.DataFrame(most_important_feature)
important_features['feautre_present_in_%_of_products']=important_features['feature_count']*100/important_features['feature_count'].max()
filtered_important_features=important_features[important_features['feautre_present_in_%_of_products'] >=10]['feature_name'].to_list()
final_scrapped_data=brand_level_data[filtered_important_features]
import analysis_function as f

2024-07-03 11:38:27.245 
  command:

    streamlit run d:\personal_git\data science\portfolio projects\Competition analysis\.venv\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-07-03 11:38:27.262 Serialization of dataframe to Arrow table was unsuccessful due to: ("Could not convert '' with type str: tried to convert to double", 'Conversion failed for column rating with type object'). Applying automatic fixes for column types to make the dataframe Arrow-compatible.


In [4]:
final_scrapped_data['ratings_count']=final_scrapped_data['ratings_count'].apply(lambda x: x if f.is_integer(x) else None).astype(float)

C:\Users\kashy\AppData\Local\Temp\ipykernel_7884\519216831.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_scrapped_data['ratings_count']=final_scrapped_data['ratings_count'].apply(lambda x: x if f.is_integer(x) else None).astype(float)


In [18]:
rating_df=final_scrapped_data.groupby('brand').agg({"ratings_count":"sum"})
rating_df.reset_index(inplace=True)
all_discription_filter=list(set(filtered_important_features).difference(set(competitor_data.columns.tolist())))
list_of_brand=['Boult','boAt','Noise']
list_of_price=[899]
features_to_analyze=final_scrapped_data[final_scrapped_data['brand'].isin(list_of_brand) & final_scrapped_data['final_selling_price'].isin(list_of_price)]
all_discription_filter.extend(['brand','fsn'])
feature_df=features_to_analyze[all_discription_filter]
feature_df=feature_df.dropna(axis=1,how='all')
feature_df['Battery Life'].nunique()
features_after_not_unique=[description for description in feature_df if feature_df[description].nunique()>1]
feature_df=feature_df[features_after_not_unique]
feature_df.groupby(['brand',features_after_not_unique[2]]).agg({"fsn":["count",list]})

fsn                                                   
                     count                                               list
brand Headphone Type                                                         
Boult In the Ear         3  [ACCGEDKJF8WR4UWE, ACCGEDKJUCXSXVGK, ACCGEDKJJ...
      True Wireless     14  [ACCGRNMZUUHD5QFY, ACCGJT2ZEDUGFNMB, ACCGTBMG7...
Noise True Wireless      4  [ACCGQSGWDGQAVUM3, ACCGQSGWFMYHSMQV, ACCGQSGWF...
boAt  In the Ear         2               [ACCGJMG6RF3MJHDX, ACCGS9ZMAXHZFU2K]
      True Wireless      3  [ACCG48F2YZNGZ8D2, ACCFSDGXX3S6DVBG, ACCFSDGXT...

In [39]:
fe_distribution=pd.DataFrame(final_scrapped_data['Color'].value_counts())
fe_distribution.reset_index(inplace=True)
fe_distribution=fe_distribution.sort_values(by='count',ascending=False).head(10)
fe_fig=px.pie(fe_distribution,values='count',names='Color')